In [1]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_mldata
import matplotlib.pyplot as plt
from sklearn import datasets, svm, metrics
from sklearn import cross_validation
#from sklearn.neural_network import MLPClassifier
from sklearn import tree
import time 
from sklearn import neighbors
from sklearn.utils import shuffle
tf.logging.set_verbosity(tf.logging.ERROR)
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

C:\Users\steja\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
mnist = fetch_mldata("MNIST original")

In [3]:
print(mnist.data.shape)

(70000, 784)


In [4]:
print(np.unique(mnist.target))

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]


In [5]:
X, y = np.float32(mnist.data[:70000]), np.float32(mnist.target[:70000])
X, y = shuffle(X,y)

In [6]:
#X, y = np.float32(mnist.data[:70000]), np.float32(mnist.target[:70000])
X_train, y_train = np.float32(X[:15000]), np.float32(y[:15000])
X_test, y_test = np.float32(X[60000:]), np.float32(y[60000:])

# PCA

In [7]:
pca = PCA(n_components=700)
X_train_pca = pca.fit_transform(X_train, y_train)
X_test_pca = pca.transform(X_test)
print('Variance covered', pca.explained_variance_ratio_.sum())
print('PCA Shape', X_train_pca.shape)

Variance covered 1.0
PCA Shape (15000, 700)


#LDA

In [8]:
lda = LinearDiscriminantAnalysis()
X_train_lda = lda.fit_transform(X_train_pca, y_train)
X_test_lda = lda.transform(X_test_pca)
print('Variance covered', lda.explained_variance_ratio_.sum())
print('PCA Shape', X_train_lda.shape)

Variance covered 1.0
PCA Shape (15000, 9)


In [9]:
start = int(round(time.time() * 1000))

In [10]:
GB = GaussianNB()
GB.fit(X_train_lda,y_train)
end = int(round(time.time() * 1000))
print("--NN fitting finished in ", (end-start), "ms--------------")
predicted=GB.predict(X_test_lda)
expected=y_test

--NN fitting finished in  265 ms--------------


In [11]:
print(cross_validation.cross_val_score(GB, X_train_lda,y_train, cv=5))

[ 0.90083195  0.88740839  0.88929643  0.89259506  0.88785047]


In [12]:
print("Classification report for kNN classifier %s:\n%s\n"
     % (GB, metrics.classification_report(expected, predicted)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

Classification report for kNN classifier GaussianNB(priors=None):
             precision    recall  f1-score   support

        0.0       0.95      0.93      0.94       987
        1.0       0.94      0.91      0.92      1091
        2.0       0.84      0.85      0.84      1017
        3.0       0.83      0.84      0.83       988
        4.0       0.87      0.87      0.87       992
        5.0       0.81      0.79      0.80       919
        6.0       0.91      0.91      0.91       985
        7.0       0.90      0.85      0.88      1055
        8.0       0.75      0.82      0.78       972
        9.0       0.81      0.83      0.82       994

avg / total       0.86      0.86      0.86     10000


Confusion matrix:
[[915   0   8   4   0  23  20   0  14   3]
 [  0 993  15   5   1   3   3   3  68   0]
 [  8  11 860  34  23  10  19   6  40   6]
 [  0   9  46 825   0  42   3  25  24  14]
 [  3   1  13   3 861   9  12   4  27  59]
 [ 10   4  14  44  11 725  21   7  68  15]
 [  9   3  18   2 

In [13]:
print("Accuracy is:",round(metrics.accuracy_score(expected,predicted)*100,2))

Accuracy is: 86.05


In [14]:
print("Test error is:",100-round(metrics.accuracy_score(expected,predicted)*100,2))

Test error is: 13.95


In [15]:
print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

Confusion matrix:
[[915   0   8   4   0  23  20   0  14   3]
 [  0 993  15   5   1   3   3   3  68   0]
 [  8  11 860  34  23  10  19   6  40   6]
 [  0   9  46 825   0  42   3  25  24  14]
 [  3   1  13   3 861   9  12   4  27  59]
 [ 10   4  14  44  11 725  21   7  68  15]
 [  9   3  18   2  10  33 897   0  13   0]
 [  3   8  20  17  20   2   0 900   4  81]
 [  6  28  21  39  11  41   6   1 800  19]
 [ 12   1   9  16  52   7   1  55  12 829]]
